In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [89]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
x = torch.tensor([1.0, 2.0, 3.0])
x = x.to(device)
print(x.device) 

cuda
cuda:0


In [87]:
torch.cuda.device_count()

2

In [3]:
!pip install torch torchvision

  Using cached torch-2.0.1-cp39-cp39-manylinux1_x86_64.whl (619.9 MB)
  Using cached torchvision-0.15.2-cp39-cp39-manylinux1_x86_64.whl (6.0 MB)
  Obtaining dependency information for filelock from https://files.pythonhosted.org/packages/00/45/ec3407adf6f6b5bf867a4462b2b0af27597a26bd3cd6e2534cb6ab029938/filelock-3.12.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 42.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 83.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 65.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 74.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 7.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 11.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [36]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image

In [37]:
gpu_id = 0  # Change this to the index of the desired GPU
torch.cuda.set_device(gpu_id)

import zipfile
zip_path = 'Updated Data.zip'
with zipfile.ZipFile(zip_path,'r') as zip_ref:
    zip_ref.extractall('RediMinds/')

In [38]:
DATADIR = "RediMinds/Data_Cleaned/"
categories = ["Anaesthesia_machine","baby_incubator","Bone_saws","C_arm","colonoscope","Curved_Mayo_Scissor","difibrillator","Electrocautery_devices","fetal_doppler","forceps","Heart_Lung_Machine","IABP","IMRT","infusion_pump","Laryngoscopes","mayfield_clamp","Needle_Biopsy_Device","phacoemulsification","Radiofrequency_Ablation_Device","Scalpel","Straight_Dissection_Clamp","Straight_Mayo_Scissor","Suction_Machine","ventilator","x_ray"]
input_size = (224, 224)

In [39]:
len(categories)

25

In [40]:
class CustomDataset(Dataset):
    def __init__(self, data_dir, categories, input_size, transform=None):
        self.data_dir = data_dir
        self.categories = categories
        self.input_size = input_size
        self.transform = transform
        self.data = []
        self.labels = []
        self.load_data()

    def load_data(self):
        for cat in self.categories:
            path = os.path.join(self.data_dir, cat)
            class_num = self.categories.index(cat)
            for img_name in os.listdir(path):
                try:
                    img_path = os.path.join(path, img_name)
                    img = Image.open(img_path).convert("RGB")
                    img = img.resize(self.input_size)
                    if self.transform:
                        img = self.transform(img)
                    self.data.append(img)
                    self.labels.append(class_num)
                except Exception as e:
                    pass

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img = self.data[idx]
        label = self.labels[idx]
        return img, label

In [41]:
transform = transforms.Compose([
    transforms.RandomResizedCrop(input_size, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    #transforms.RandomRotation(15),
    #transforms.ColorJitter(brightness=0.2, contrast = 0.25, saturation = 0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.2, 0.2, 0.2))
])

dataset = CustomDataset(DATADIR, categories, input_size, transform=transform)

train_size = int(0.70 * len(dataset))
validation_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - validation_size
train_dataset, validation_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size,validation_size, test_size])

In [42]:
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class CustomMobileNet(nn.Module):
    def __init__(self, num_classes, fine_tune=True):
        super(CustomMobileNet, self).__init__()
        self.base_model = models.mobilenet_v3_large(pretrained='imagenet')
        if not fine_tune:
            for param in self.base_model.parameters():
                param.requires_grad = False

        num_features = self.base_model.classifier[0].in_features
        self.base_model.classifier = nn.Sequential(
            nn.Conv2d(num_features, 128, kernel_size=1),
            nn.AdaptiveAvgPool2d(1))
        self.cnn1 = nn.Conv2d(960,2048,kernel_size=3, padding=1)
        self.dropout1 = nn.Dropout(0.5)
        #self.cnn2 = nn.Conv2d(1024,512,kernel_size = 3, padding=1)
        #self.dropout5 = nn.Dropout(0.3)
        self.fc1 = nn.Linear(2048, 2048)
        self.dropout2 = nn.Dropout(0.5)
        self.fc7 = nn.Linear(2048, 1024)
        self.dropout5 = nn.Dropout(0.5)
        self.fc8 = nn.Linear(1024,1024)
        self.dropout6 = nn.Dropout(0.4)
        self.fc6 = nn.Linear(1024, num_classes)
        # self.fc2 = nn.Linear(128, 64)
        # self.dropout3 = nn.Dropout(0.4)
        # self.fc3 = nn.Linear(64, 32)
        # self.dropout4 = nn.Dropout(0.2)
        # self.fc4 = nn.Linear(32, 16)
        # self.fc5 = nn.Linear(16, num_classes)

    def forward(self, x):
        x = self.base_model.features(x)
        x = self.cnn1(x)
        x = torch.relu(x)
        x = self.dropout1(x)
        #x = self.cnn2(x)
        #x = torch.relu(x)
        x = torch.mean(x, dim=[2, 3])  # Global average pooling
        x = x.view(x.size(0), -1)
        #x = self.dropout5(x)
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.dropout2(x)
        x = self.fc7(x)
        x = torch.relu(x)
        x = self.dropout5(x)
        x = self.fc8(x)
        x = torch.relu(x)
        x = self.dropout6(x)
        x = self.fc6(x)
        # x = torch.relu(x)
        # x = self.fc2(x)
        # x = torch.relu(x)
        # x = self.dropout3(x)
        # x = self.fc3(x)
        # x = torch.relu(x)
        # # x = self.dropout4(x)
        # x = self.fc4(x)
        # x = torch.relu(x)
        # x = self.fc5(x)
        return x

In [93]:
class CustomMobileNet(nn.Module):
    def __init__(self, num_classes,fine_tune = False):
        super(CustomMobileNet, self).__init__()
        self.base_model = models.mobilenet_v3_large(pretrained=models.MobileNet_V3_Large_Weights.IMAGENET1K_V2)
        self.base_model = models.mobilenet_v3_large(pretrained=models.MobileNet_V3_Large_Weights.IMAGENET1K_V2)
        

        if not fine_tune:
            # Freeze all layers except the last few
            for param in self.base_model.parameters():
                param.requires_grad = False

            # Identify the last few layers of the base model and set requires_grad to True
            for param in self.base_model.features[-3:].parameters():
                param.requires_grad = True
                
        num_features = self.base_model.classifier[-1].in_features
        self.base_model.classifier[-1] = nn.Sequential(
            nn.Conv2d(num_features, 128, kernel_size=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d(1))
        self.cnn1 = nn.Conv2d(960,4098,kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(4098)
        self.dropout1 = nn.Dropout(0.6)
        # self.cnn2 = nn.Conv2d(1024, 1024,kernel_size = 3, padding = 1)
        # self.dropout2 = nn.Dropout(0.6)
        # self.cnn3 = nn.Conv2d(1024, 512,kernel_size = 3, padding = 1)
        # self.dropout3 = nn.Dropout(0.5)
        # self.bn2 = nn.BatchNorm2d(512)
        self.cnn4 = nn.Conv2d(4098, 4098,kernel_size = 3, padding = 1)
        self.bn4 = nn.BatchNorm2d(4098)
        self.dropout4 = nn.Dropout(0.5)
        self.cnn5 = nn.Conv2d(4098, 2048,kernel_size = 3, padding = 1)
        self.dropout5 = nn.Dropout(0.6)
        self.bn3 = nn.BatchNorm2d(2048)
        self.avgpooling = nn.AdaptiveAvgPool2d((1, 1))
        self.base_model = nn.Sequential(*list(self.base_model.children())[:-1])
        # self.fc1 = nn.Linear(128, 64)
        # self.dropout6 = nn.Dropout(0.4)
        self.fc2 = nn.Linear(2048,2048)
        self.bnl = nn.BatchNorm1d(2048)
        self.dropout7 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(2048, num_classes)

    def forward(self, x):
        x = self.base_model(x)
        x = self.cnn1(x)
        x = self.bn1(x)
        x = torch.relu(x)
        x = self.dropout1(x)
        # x = self.cnn2(x)
        # x = torch.relu(x)
        # x = self.dropout2(x)
        # x = self.cnn3(x)
        # x = self.bn2(x)
        # x = torch.relu(x)
        # x = self.dropout3(x)
        # x = self.cnn4(x)
        # x = self.bn4(x)
        # x = torch.relu(x)
        # x = self.dropout4(x)
        x = self.cnn5(x)
        x = self.bn3(x)
        x = torch.relu(x)
        x = self.dropout5(x)
        x = self.avgpooling(x)
        #x = torch.mean(x, dim=[2, 3])  # Global average pooling
        x = x.view(x.size(0), -1)
        # x = self.fc1(x)
        # x = torch.relu(x)
        # x = self.dropout6(x)
        x = self.fc2(x)
        x = self.bnl(x)
        x = torch.relu(x)
        x = self.dropout7(x)
        x = self.fc3(x)
        return x

weight_enum = torch.hub.load("pytorch/vision", "get_model_weights", name="mobilenet_v3_large")
print([weight for weight in weight_enum])

In [94]:
from torch.optim.lr_scheduler import StepLR
num_classes = len(categories)
model = CustomMobileNet(num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.005,weight_decay=1e-6)
scheduler = StepLR(optimizer, step_size = 20, gamma=0.1)

In [95]:
for name, param in model.named_parameters():
    print(f"{name}: requires_grad={param.requires_grad}")

base_model.0.0.0.weight: requires_grad=False
base_model.0.0.1.weight: requires_grad=False
base_model.0.0.1.bias: requires_grad=False
base_model.0.1.block.0.0.weight: requires_grad=False
base_model.0.1.block.0.1.weight: requires_grad=False
base_model.0.1.block.0.1.bias: requires_grad=False
base_model.0.1.block.1.0.weight: requires_grad=False
base_model.0.1.block.1.1.weight: requires_grad=False
base_model.0.1.block.1.1.bias: requires_grad=False
base_model.0.2.block.0.0.weight: requires_grad=False
base_model.0.2.block.0.1.weight: requires_grad=False
base_model.0.2.block.0.1.bias: requires_grad=False
base_model.0.2.block.1.0.weight: requires_grad=False
base_model.0.2.block.1.1.weight: requires_grad=False
base_model.0.2.block.1.1.bias: requires_grad=False
base_model.0.2.block.2.0.weight: requires_grad=False
base_model.0.2.block.2.1.weight: requires_grad=False
base_model.0.2.block.2.1.bias: requires_grad=False
base_model.0.3.block.0.0.weight: requires_grad=False
base_model.0.3.block.0.1.weig

In [ ]:
train_acc_history = []
train_loss_history = []
val_acc_history = []
val_loss_history = []

num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    train_loss, train_corrects = 0.0, 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        train_corrects += torch.sum(preds == labels).item()

    train_loss = train_loss / len(train_dataset)
    train_acc = train_corrects / len(train_dataset)
    train_acc_history.append(train_acc)
    train_loss_history.append(train_loss)

    model.eval()
    val_loss, val_corrects = 0.0, 0.0

    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            val_corrects += torch.sum(preds == labels).item()

    val_loss = val_loss / len(validation_dataset)
    val_acc = val_corrects / len(validation_dataset)

    val_acc_history.append(val_acc)
    val_loss_history.append(val_loss)

    scheduler.step()

    print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

Epoch 1/50 | Train Loss: 3.1270 | Train Acc: 0.2820 | Val Loss: 194.5615 | Val Acc: 0.0615
Epoch 2/50 | Train Loss: 2.4702 | Train Acc: 0.4620 | Val Loss: 18.7272 | Val Acc: 0.1746
Epoch 3/50 | Train Loss: 2.1446 | Train Acc: 0.5314 | Val Loss: 16.8683 | Val Acc: 0.3939
Epoch 4/50 | Train Loss: 1.6966 | Train Acc: 0.5966 | Val Loss: 2.0972 | Val Acc: 0.5656
Epoch 5/50 | Train Loss: 1.4978 | Train Acc: 0.6555 | Val Loss: 7.4148 | Val Acc: 0.4749
Epoch 6/50 | Train Loss: 1.1915 | Train Acc: 0.7078 | Val Loss: 1.2464 | Val Acc: 0.6830
Epoch 7/50 | Train Loss: 0.9551 | Train Acc: 0.7641 | Val Loss: 1.2442 | Val Acc: 0.7151
Epoch 8/50 | Train Loss: 0.6976 | Train Acc: 0.7952 | Val Loss: 0.8147 | Val Acc: 0.7654


In [ ]:
# Plot training and validation accuracy
import matplotlib.pyplot as plt

plt.plot(range(1, num_epochs+1), train_acc_history, label='Train')
plt.plot(range(1, num_epochs+1), val_acc_history, label='Validation')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot training and validation loss
plt.plot(range(1, num_epochs+1), train_loss_history, label='Train')
plt.plot(range(1, num_epochs+1), val_loss_history, label='Validation')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
model.eval()
test_loss, test_corrects = 0.0, 0.0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        test_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        test_corrects += torch.sum(preds == labels).item()

test_loss = test_loss / len(test_dataset)
test_acc = test_corrects / len(test_dataset)

print(f"Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.4f}")

---

model.eval()
test_loss, test_corrects = 0.0, 0.0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        test_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        test_corrects += torch.sum(preds == labels).item()

test_loss = test_loss / len(test_dataset)
test_acc = test_corrects / len(test_dataset)

print(f"Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.4f}")

In [79]:
torch.save(model.state_dict(),"model_torch_mobilenet_2.pth")

In [23]:
def calculate_accuracy_and_loss(model, loader):
    """
    Calculates the accuracy and loss for the given model and loader.

    Args:
        model: The model to evaluate.
        loader: The data loader to use.

    Returns:
        The accuracy and loss values.
    """
    model.eval()
    corrects = 0
    total = 0
    loss = 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss += criterion(outputs, labels).item() * images.size(0)

            _, preds = torch.max(outputs, 1)
            corrects += torch.sum(preds == labels).item()
            total += labels.size(0)

    return corrects / total, loss / total

In [ ]:
import matplotlib.pyplot as plt

train_accuracy_values = []
test_accuracy_values = []
train_loss_values = []
test_loss_values = []

num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    train_loss, train_corrects = 0.0, 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        train_corrects += torch.sum(preds == labels).item()

        train_accuracy_values.append(train_corrects / len(train_dataset))
        train_loss_values.append(train_loss / len(train_dataset))

        test_accuracy, test_loss = calculate_accuracy_and_loss(model, test_loader)
        test_accuracy_values.append(test_accuracy)
        test_loss_values.append(test_loss)

    train_loss = train_loss / len(train_dataset)
    train_acc = train_corrects / len(train_dataset)

    print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.4f}")

plt.figure()
plt.plot(train_accuracy_values, label="Train Accuracy")
plt.plot(test_accuracy_values, label="Test Accuracy")
plt.legend()
plt.title("Accuracy vs. Epoch")
plt.show()

plt.figure()
plt.plot(train_loss_values, label="Train Loss")
plt.plot(test_loss_values, label="Test Loss")
plt.legend()
plt.title("Loss vs. Epoch")
plt.show()

Epoch 1/50 | Train Loss: 3.4058 | Train Acc: 0.0711
Test Loss: 3.2241 | Test Acc: 0.8341
Epoch 2/50 | Train Loss: 3.2104 | Train Acc: 0.0495
Test Loss: 3.2266 | Test Acc: 0.8341
Epoch 3/50 | Train Loss: 3.2094 | Train Acc: 0.0509
Test Loss: 3.2276 | Test Acc: 0.8341
Epoch 4/50 | Train Loss: 3.1975 | Train Acc: 0.0513
Test Loss: 3.2225 | Test Acc: 0.8341
Epoch 5/50 | Train Loss: 3.1950 | Train Acc: 0.0505
Test Loss: 3.2245 | Test Acc: 0.8341
Epoch 6/50 | Train Loss: 3.1912 | Train Acc: 0.0493
Test Loss: 3.2241 | Test Acc: 0.8341
Epoch 7/50 | Train Loss: 3.1908 | Train Acc: 0.0509
Test Loss: 3.2252 | Test Acc: 0.8341
Epoch 8/50 | Train Loss: 3.1940 | Train Acc: 0.0515
Test Loss: 3.2296 | Test Acc: 0.8341
Epoch 9/50 | Train Loss: 3.1926 | Train Acc: 0.0515
Test Loss: 3.2229 | Test Acc: 0.8341
Epoch 10/50 | Train Loss: 3.1902 | Train Acc: 0.0499
Test Loss: 3.2217 | Test Acc: 0.8341
Epoch 11/50 | Train Loss: 3.1855 | Train Acc: 0.0519
Test Loss: 3.2241 | Test Acc: 0.8341
Epoch 12/50 | Train